In [1]:
import os
import sys
sys.path.append('../..')
import utils
import google.generativeai as genai

# Configure your API key (replace with your actual API key)
genai.configure(api_key="")

def get_completion_from_messages(messages, model="gemini-1.5-flash"):
    
    try:
        # Create an instance of the GenerativeModel
        model_instance = genai.GenerativeModel(model_name=model)
        
        # Generate content using the model with structured input
        response = model_instance.generate_content(
            contents=[{
                "role": "user",
                "parts": [{"text": msg["content"]} for msg in messages]
            }]
        )
        
        # Extract and return the generated content
        return response.text  # Adjust based on actual response structure

    except Exception as e:
        print(f"An error occurred: {e}")
        return None

In [2]:
products_and_category = utils.get_products_and_category()
products_and_category

{'Computers and Laptops': ['TechPro Ultrabook',
  'BlueWave Gaming Laptop',
  'PowerLite Convertible',
  'TechPro Desktop',
  'BlueWave Chromebook'],
 'Smartphones and Accessories': ['SmartX ProPhone',
  'MobiTech PowerCase',
  'SmartX MiniPhone',
  'MobiTech Wireless Charger',
  'SmartX EarBuds'],
 'Televisions and Home Theater Systems': ['CineView 4K TV',
  'SoundMax Home Theater',
  'CineView 8K TV',
  'SoundMax Soundbar',
  'CineView OLED TV'],
 'Gaming Consoles and Accessories': ['GameSphere X',
  'ProGamer Controller',
  'GameSphere Y',
  'ProGamer Racing Wheel',
  'GameSphere VR Headset'],
 'Audio Equipment': ['AudioPhonic Noise-Canceling Headphones',
  'WaveSound Bluetooth Speaker',
  'AudioPhonic True Wireless Earbuds',
  'WaveSound Soundbar',
  'AudioPhonic Turntable'],
 'Cameras and Camcorders': ['FotoSnap DSLR Camera',
  'ActionCam 4K',
  'FotoSnap Mirrorless Camera',
  'ZoomMaster Camcorder',
  'FotoSnap Instant Camera']}

In [3]:
def find_category_and_product_v1(user_input,products_and_category):

    delimiter = "####"
    system_message = f"""
    You will be provided with customer service queries. \
    The customer service query will be delimited with {delimiter} characters.
    Output a python list of json objects, where each object has the following format:
        'category': <one of Computers and Laptops, Smartphones and Accessories, Televisions and Home Theater Systems, \
    Gaming Consoles and Accessories, Audio Equipment, Cameras and Camcorders>,
    AND
        'products': <a list of products that must be found in the allowed products below>


    Where the categories and products must be found in the customer service query.
    If a product is mentioned, it must be associated with the correct category in the allowed products list below.
    If no products or categories are found, output an empty list.
    

    List out all products that are relevant to the customer service query based on how closely it relates
    to the product name and product category.
    Do not assume, from the name of the product, any features or attributes such as relative quality or price.

    The allowed products are provided in JSON format.
    The keys of each item represent the category.
    The values of each item is a list of products that are within that category.
    Allowed products: {products_and_category}
    

    """
    
    few_shot_user_1 = """I want the most expensive computer."""
    few_shot_assistant_1 = """ 
    [{'category': 'Computers and Laptops', \
'products': ['TechPro Ultrabook', 'BlueWave Gaming Laptop', 'PowerLite Convertible', 'TechPro Desktop', 'BlueWave Chromebook']}]
    """
    
    messages =  [  
    {'role':'system', 'content': system_message},    
    {'role':'user', 'content': f"{delimiter}{few_shot_user_1}{delimiter}"},  
    {'role':'assistant', 'content': few_shot_assistant_1 },
    {'role':'user', 'content': f"{delimiter}{user_input}{delimiter}"},  
    ] 
    return get_completion_from_messages(messages)


In [4]:
customer_msg_0 = f"""Which TV can I buy if I'm on a budget?"""

products_by_category_0 = find_category_and_product_v1(customer_msg_0,
                                                      products_and_category)
print(products_by_category_0)

```json
[{'category': 'Televisions and Home Theater Systems', 'products': ['CineView 4K TV', 'SoundMax Home Theater', 'CineView 8K TV', 'SoundMax Soundbar', 'CineView OLED TV']}]
```


In [5]:
customer_msg_1 = f"""I need a charger for my smartphone"""

products_by_category_1 = find_category_and_product_v1(customer_msg_1,
                                                      products_and_category)
print(products_by_category_1)

```json
[{'category': 'Smartphones and Accessories', 'products': ['MobiTech PowerCase', 'MobiTech Wireless Charger']}]
```


In [6]:
customer_msg_2 = f"""
What computers do you have?"""

products_by_category_2 = find_category_and_product_v1(customer_msg_2,
                                                      products_and_category)
products_by_category_2

"[{'category': 'Computers and Laptops', 'products': ['TechPro Ultrabook', 'BlueWave Gaming Laptop', 'PowerLite Convertible', 'TechPro Desktop', 'BlueWave Chromebook']}]\n"

In [7]:
customer_msg_3 = f"""
tell me about the smartx pro phone and the fotosnap camera, the dslr one.
Also, what TVs do you have?"""

products_by_category_3 = find_category_and_product_v1(customer_msg_3,
                                                      products_and_category)
print(products_by_category_3)

```json
[{'category': 'Smartphones and Accessories', 'products': ['SmartX ProPhone']}, {'category': 'Cameras and Camcorders', 'products': ['FotoSnap DSLR Camera']}, {'category': 'Televisions and Home Theater Systems', 'products': ['CineView 4K TV', 'CineView 8K TV', 'CineView OLED TV']}]
```


In [8]:
customer_msg_4 = f"""
tell me about the CineView TV, the 8K one, Gamesphere console, the X one.
I'm on a budget, what computers do you have?"""

products_by_category_4 = find_category_and_product_v1(customer_msg_4,
                                                      products_and_category)
print(products_by_category_4)

[{'category': 'Televisions and Home Theater Systems', 'products': ['CineView 8K TV']}, {'category': 'Gaming Consoles and Accessories', 'products': ['GameSphere X']}, {'category': 'Computers and Laptops', 'products': ['BlueWave Chromebook', 'TechPro Desktop', 'PowerLite Convertible', 'BlueWave Gaming Laptop', 'TechPro Ultrabook']}]



In [9]:
def find_category_and_product_v2(user_input,products_and_category):
    """
    Added: Do not output any additional text that is not in JSON format.
    Added a second example (for few-shot prompting) where user asks for 
    the cheapest computer. In both few-shot examples, the shown response 
    is the full list of products in JSON only.
    """
    delimiter = "####"
    system_message = f"""
    You will be provided with customer service queries. \
    The customer service query will be delimited with {delimiter} characters.
    Output a python list of json objects, where each object has the following format:
        'category': <one of Computers and Laptops, Smartphones and Accessories, Televisions and Home Theater Systems, \
    Gaming Consoles and Accessories, Audio Equipment, Cameras and Camcorders>,
    AND
        'products': <a list of products that must be found in the allowed products below>
    Do not output any additional text that is not in JSON format.
    Do not write any explanatory text after outputting the requested JSON.


    Where the categories and products must be found in the customer service query.
    If a product is mentioned, it must be associated with the correct category in the allowed products list below.
    If no products or categories are found, output an empty list.
    

    List out all products that are relevant to the customer service query based on how closely it relates
    to the product name and product category.
    Do not assume, from the name of the product, any features or attributes such as relative quality or price.

    The allowed products are provided in JSON format.
    The keys of each item represent the category.
    The values of each item is a list of products that are within that category.
    Allowed products: {products_and_category}
    

    """
    
    few_shot_user_1 = """I want the most expensive computer. What do you recommend?"""
    few_shot_assistant_1 = """ 
    [{'category': 'Computers and Laptops', \
'products': ['TechPro Ultrabook', 'BlueWave Gaming Laptop', 'PowerLite Convertible', 'TechPro Desktop', 'BlueWave Chromebook']}]
    """
    
    few_shot_user_2 = """I want the most cheapest computer. What do you recommend?"""
    few_shot_assistant_2 = """ 
    [{'category': 'Computers and Laptops', \
'products': ['TechPro Ultrabook', 'BlueWave Gaming Laptop', 'PowerLite Convertible', 'TechPro Desktop', 'BlueWave Chromebook']}]
    """
    
    messages =  [  
    {'role':'system', 'content': system_message},    
    {'role':'user', 'content': f"{delimiter}{few_shot_user_1}{delimiter}"},  
    {'role':'assistant', 'content': few_shot_assistant_1 },
    {'role':'user', 'content': f"{delimiter}{few_shot_user_2}{delimiter}"},  
    {'role':'assistant', 'content': few_shot_assistant_2 },
    {'role':'user', 'content': f"{delimiter}{user_input}{delimiter}"},  
    ] 
    return get_completion_from_messages(messages)


In [10]:
customer_msg_3 = f"""
tell me about the smartx pro phone and the fotosnap camera, the dslr one.
Also, what TVs do you have?"""

products_by_category_3 = find_category_and_product_v2(customer_msg_3,
                                                      products_and_category)
print(products_by_category_3)

[{'category': 'Smartphones and Accessories', 'products': ['SmartX ProPhone']}, {'category': 'Cameras and Camcorders', 'products': ['FotoSnap DSLR Camera']}, {'category': 'Televisions and Home Theater Systems', 'products': ['CineView 4K TV', 'SoundMax Home Theater', 'CineView 8K TV', 'SoundMax Soundbar', 'CineView OLED TV']}]



In [11]:
customer_msg_31 = f"""
want the most cheapest computer?"""

products_by_category_31 = find_category_and_product_v2(customer_msg_31,
                                                      products_and_category)
print(products_by_category_31)

[{'category': 'Computers and Laptops', 'products': ['TechPro Ultrabook', 'BlueWave Gaming Laptop', 'PowerLite Convertible', 'TechPro Desktop', 'BlueWave Chromebook']}]



In [12]:
customer_msg_0 = f"""Which TV can I buy if I'm on a budget?"""

products_by_category_0 = find_category_and_product_v2(customer_msg_0,
                                                      products_and_category)
print(products_by_category_0)

[{'category': 'Televisions and Home Theater Systems', 'products': ['CineView 4K TV', 'SoundMax Home Theater', 'CineView 8K TV', 'SoundMax Soundbar', 'CineView OLED TV']}]



In [13]:
msg_ideal_pairs_set = [
    
    # eg 0
    {'customer_msg':"""Which TV can I buy if I'm on a budget?""",
     'ideal_answer':{
        'Televisions and Home Theater Systems':set(
            ['CineView 4K TV', 'SoundMax Home Theater', 'CineView 8K TV', 'SoundMax Soundbar', 'CineView OLED TV']
        )}
    },

    # eg 1
    {'customer_msg':"""I need a charger for my smartphone""",
     'ideal_answer':{
        'Smartphones and Accessories':set(
            ['MobiTech PowerCase', 'MobiTech Wireless Charger', 'SmartX EarBuds']
        )}
    },
    # eg 2
    {'customer_msg':f"""What computers do you have?""",
     'ideal_answer':{
           'Computers and Laptops':set(
               ['TechPro Ultrabook', 'BlueWave Gaming Laptop', 'PowerLite Convertible', 'TechPro Desktop', 'BlueWave Chromebook'
               ])
                }
    },

    # eg 3
    {'customer_msg':f"""tell me about the smartx pro phone and \
    the fotosnap camera, the dslr one.\
    Also, what TVs do you have?""",
     'ideal_answer':{
        'Smartphones and Accessories':set(
            ['SmartX ProPhone']),
        'Cameras and Camcorders':set(
            ['FotoSnap DSLR Camera']),
        'Televisions and Home Theater Systems':set(
            ['CineView 4K TV', 'SoundMax Home Theater','CineView 8K TV', 'SoundMax Soundbar', 'CineView OLED TV'])
        }
    }, 
    
    # eg 4
    {'customer_msg':"""tell me about the CineView TV, the 8K one, Gamesphere console, the X one.
I'm on a budget, what computers do you have?""",
     'ideal_answer':{
        'Televisions and Home Theater Systems':set(
            ['CineView 8K TV']),
        'Gaming Consoles and Accessories':set(
            ['GameSphere X']),
        'Computers and Laptops':set(
            ['TechPro Ultrabook', 'BlueWave Gaming Laptop', 'PowerLite Convertible', 'TechPro Desktop', 'BlueWave Chromebook'])
        }
    },
    
    # eg 5
    {'customer_msg':f"""What smartphones do you have?""",
     'ideal_answer':{
           'Smartphones and Accessories':set(
               ['SmartX ProPhone', 'MobiTech PowerCase', 'SmartX MiniPhone', 'MobiTech Wireless Charger', 'SmartX EarBuds'
               ])
                    }
    },
    # eg 6
    {'customer_msg':f"""I'm on a budget.  Can you recommend some smartphones to me?""",
     'ideal_answer':{
        'Smartphones and Accessories':set(
            ['SmartX EarBuds', 'SmartX MiniPhone', 'MobiTech PowerCase', 'SmartX ProPhone', 'MobiTech Wireless Charger']
        )}
    },

    # eg 7 # this will output a subset of the ideal answer
    {'customer_msg':f"""What Gaming consoles would be good for my friend who is into racing games?""",
     'ideal_answer':{
        'Gaming Consoles and Accessories':set([
            'GameSphere X',
            'ProGamer Controller',
            'GameSphere Y',
            'ProGamer Racing Wheel',
            'GameSphere VR Headset'
     ])}
    },
    # eg 8
    {'customer_msg':f"""What could be a good present for my videographer friend?""",
     'ideal_answer': {
        'Cameras and Camcorders':set([
        'FotoSnap DSLR Camera', 'ActionCam 4K', 'FotoSnap Mirrorless Camera', 'ZoomMaster Camcorder', 'FotoSnap Instant Camera'
        ])}
    },
    
    # eg 9
    {'customer_msg':f"""I would like a hot tub time machine.""",
     'ideal_answer': []
    }
    
]


######## CODE EXPLANATION
here AI generated response 
response = "[{'category': 'Computers and Laptops', 'products': ['TechPro Ultrabook', 'BlueWave Gaming Laptop']}, {'category': 'Gaming Consoles', 'products': ['GameSphere X']}]"
Since this string uses single quotes instead of double quotes (which are required for valid JSON) we need to replace single quotes with double quotes:

Next,
json_like_str = response.replace("'", '"')

Next,
Now we can parse this modified string into a Python data structure:

l_of_d = json.loads(json_like_str)

Resulting Parsed Output
After parsing, l_of_d will contain:

l_of_d = [
    {'category': 'Computers and Laptops', 'products': ['TechPro Ultrabook', 'BlueWave Gaming Laptop']},
    {'category': 'Gaming Consoles', 'products': ['GameSphere X']}
]

Next,
Now that we have our parsed output in l_of_d, we can use it for evaluation against an ideal output:ideal = {
    'Computers and Laptops': ['TechPro Ultrabook', 'BlueWave Gaming Laptop', 'PowerLite Convertible'],
    'Gaming Consoles': ['GameSphere X']Next,
}

correctThis line initializes a variable named correct to 0. This variable will be used to count how many categories in the parsed response (l_of_d) match those in the ideal output.

Next,
cat = d.get('category')  # 4. Checking for valid categories and products.
prod_l = d.get('products')cat: This variable retrieves the value associated with the key 'category' from the dictionary d. It represents the name of the category (e.g., "Computers and Laptops").
prod_l: This variable retrieves the value associated with the key 'products', which is expected to be a list of products belonging to that category (e.g., ['TechPro Ultrabook', 'BlueWave Gaming Laptop']).
Next,
prod_set = set(prod_l)

converts the list of products (prod_l) into a set (prod_set).
Using a set allows for easier comparison operations, such as checking for equality, subsets, or supersets.


 = 0
n


In [30]:
import json
def eval_response_with_ideal(response,
                              ideal,
                              debug=False):
    
    if debug:
        print("response")
        print(response)                          
    
    # json.loads() expects double quotes, not single quotes                  

    json_like_str = response.replace("'",'"')
    
    # parse into a list of dictionaries
    l_of_d = json.loads(json_like_str)#parsed output refers to the result obtained after converting a JSON-like string  into a usable data structure in Python, such as a list or dictionary.
    
    # special case when response is empty list
    if l_of_d == [] and ideal == []:
        return 1
    
    # otherwise, response is empty 
    # or ideal should be empty, there's a mismatch
    elif l_of_d == [] or ideal == []:
        return 0
    
    correct = 0                                                                  #1.Initializing a counter for correct matches.                                                                
    if debug:                                                                    #2.Printing debug information if enabled.
        print("l_of_d is")
        print(l_of_d)
    for d in l_of_d:                                                              #3.Iterating through each item in the parsed response.

        cat = d.get('category')                                                   #4.Checking for valid categories and products.

        prod_l = d.get('products')
        if cat and prod_l:
            # convert list to set for comparison
            prod_set = set(prod_l)
            # get ideal set of products
            ideal_cat = ideal.get(cat)
            if ideal_cat:
                prod_set_ideal = set(ideal.get(cat))
            else:
                if debug:
                    print(f"did not find category {cat} in ideal")
                    print(f"ideal: {ideal}")
                continue
                
            if debug:                                                              #5.Comparing sets of products from both actual and ideal outputs.

                print("prod_set\n",prod_set)
                print()
                print("prod_set_ideal\n",prod_set_ideal)

            if prod_set == prod_set_ideal:
                if debug:
                    print("correct")
                correct +=1
            else:
                print("incorrect")
                print(f"prod_set: {prod_set}")
                print(f"prod_set_ideal: {prod_set_ideal}")
                if prod_set <= prod_set_ideal:
                    print("response is a subset of the ideal answer")
                elif prod_set >= prod_set_ideal:
                    print("response is a superset of the ideal answer")

    # count correct over total number of items in list
    pc_correct = correct / len(l_of_d)                                         #Calculating and returning an accuracy percentage based on matches.
        
    return pc_correct

In [31]:
print(f'Customer message: {msg_ideal_pairs_set[7]["customer_msg"]}')
print(f'Ideal answer: {msg_ideal_pairs_set[7]["ideal_answer"]}')


Customer message: What Gaming consoles would be good for my friend who is into racing games?
Ideal answer: {'Gaming Consoles and Accessories': {'GameSphere X', 'GameSphere Y', 'ProGamer Racing Wheel', 'ProGamer Controller', 'GameSphere VR Headset'}}


In [32]:
response = find_category_and_product_v2(msg_ideal_pairs_set[7]["customer_msg"],
                                         products_and_category)
print(f'Response: {response}')

eval_response_with_ideal(response,
                              msg_ideal_pairs_set[7]["ideal_answer"])                          #perfect match 1.0

Response: [{'category': 'Gaming Consoles and Accessories', 'products': ['GameSphere X', 'GameSphere Y', 'ProGamer Racing Wheel']}]

incorrect
prod_set: {'GameSphere Y', 'ProGamer Racing Wheel', 'GameSphere X'}
prod_set_ideal: {'GameSphere Y', 'ProGamer Racing Wheel', 'ProGamer Controller', 'GameSphere X', 'GameSphere VR Headset'}
response is a subset of the ideal answer


0.0

In [33]:
response = find_category_and_product_v2(msg_ideal_pairs_set[9]["customer_msg"],
                                         products_and_category)
print(f'Response: {response}')

eval_response_with_ideal(response,
                              msg_ideal_pairs_set[9]["ideal_answer"])                      #question is not suitable so 1

Response: []



1

In [34]:
response = find_category_and_product_v2(msg_ideal_pairs_set[6]["customer_msg"],
                                         products_and_category)
print(f'Response: {response}')

eval_response_with_ideal(response,
                              msg_ideal_pairs_set[6]["ideal_answer"])

Response: [{'category': 'Smartphones and Accessories', 'products': ['SmartX ProPhone', 'SmartX MiniPhone']}]

incorrect
prod_set: {'SmartX MiniPhone', 'SmartX ProPhone'}
prod_set_ideal: {'MobiTech Wireless Charger', 'SmartX EarBuds', 'SmartX ProPhone', 'SmartX MiniPhone', 'MobiTech PowerCase'}
response is a subset of the ideal answer


0.0

In [35]:
response = find_category_and_product_v2(msg_ideal_pairs_set[0]["customer_msg"],
                                         products_and_category)
print(f'Response: {response}')

eval_response_with_ideal(response,
                              msg_ideal_pairs_set[0]["ideal_answer"])

Response: [{'category': 'Televisions and Home Theater Systems', 'products': ['CineView 4K TV', 'SoundMax Home Theater', 'CineView 8K TV', 'SoundMax Soundbar', 'CineView OLED TV']}]



1.0

In [36]:
# Note, this will not work if any of the api calls time out
score_accum = 0
for i, pair in enumerate(msg_ideal_pairs_set):
    print(f"example {i}")
    
    customer_msg = pair['customer_msg']
    ideal = pair['ideal_answer']
    
    # print("Customer message",customer_msg)
    # print("ideal:",ideal)
    response = find_category_and_product_v2(customer_msg,
                                                      products_and_category)
    
    
    # print("products_by_category",products_by_category)
    score = eval_response_with_ideal(response,ideal,debug=False)
    print(f"{i}: {score}")
    score_accum += score
    

n_examples = len(msg_ideal_pairs_set)
fraction_correct = score_accum / n_examples
print(f"Fraction correct out of {n_examples}: {fraction_correct}")

example 0
0: 1.0
example 1
incorrect
prod_set: {'MobiTech Wireless Charger', 'MobiTech PowerCase'}
prod_set_ideal: {'MobiTech Wireless Charger', 'SmartX EarBuds', 'MobiTech PowerCase'}
response is a subset of the ideal answer
1: 0.0
example 2
2: 1.0
example 3
3: 1.0
example 4
4: 1.0
example 5
incorrect
prod_set: {'SmartX MiniPhone', 'SmartX ProPhone'}
prod_set_ideal: {'MobiTech Wireless Charger', 'SmartX EarBuds', 'SmartX ProPhone', 'SmartX MiniPhone', 'MobiTech PowerCase'}
response is a subset of the ideal answer
5: 0.0
example 6
incorrect
prod_set: {'SmartX MiniPhone', 'SmartX ProPhone'}
prod_set_ideal: {'MobiTech Wireless Charger', 'SmartX EarBuds', 'SmartX ProPhone', 'SmartX MiniPhone', 'MobiTech PowerCase'}
response is a subset of the ideal answer
6: 0.0
example 7
incorrect
prod_set: {'GameSphere Y', 'ProGamer Racing Wheel', 'GameSphere X'}
prod_set_ideal: {'GameSphere Y', 'ProGamer Racing Wheel', 'ProGamer Controller', 'GameSphere X', 'GameSphere VR Headset'}
response is a subset

##### EXAAMPLE TO UNDERSTAND

In [ ]:
import json

def eval_response_with_ideal(response, ideal):
    # Simulate parsing the AI's response
    try:
        # Replace single quotes with double quotes for valid JSON
        json_like_str = response.replace("'", '"')
        l_of_d = json.loads(json_like_str)  # Parse into a list of dictionaries
    except json.JSONDecodeError:
        print("Invalid JSON format in response.")
        return 0  # Return 0 for invalid response format

    # Check if both the parsed response and ideal output are empty
    if l_of_d == [] and ideal == []:
        return 1  # Perfect match: both are empty
    
    # Check if either the parsed response or ideal output is empty
    elif l_of_d == [] or ideal == []:
        return 0  # Mismatch: one is empty while the other is not

    # Additional evaluation logic can go here...
    
    return -1  # Placeholder for further evaluation (not implemented)

# Example usage:

# Case 1: Both responses are empty
response_1 = "[]"
ideal_1 = []
result_1 = eval_response_with_ideal(response_1, ideal_1)
print(f"Result for Case 1: {result_1}")  # Output: 1

# Case 2: AI response is empty, but ideal is not
response_2 = "[]"
ideal_2 = {'category': 'Computers', 'products': ['TechPro Ultrabook']}
result_2 = eval_response_with_ideal(response_2, ideal_2)
print(f"Result for Case 2: {result_2}")  # Output: 0

# Case 3: Ideal is empty, but AI response is not
response_3 = "[{'category': 'Computers', 'products': ['TechPro Ultrabook']}]"
ideal_3 = []
result_3 = eval_response_with_ideal(response_3, ideal_3)
print(f"Result for Case 3: {result_3}")  # Output: 0

# Case 4: Both responses contain data (not handled in this example)
response_4 = "[{'category': 'Computers', 'products': ['TechPro Ultrabook']}]"
ideal_4 = {'Computers': ['TechPro Ultrabook']}
result_4 = eval_response_with_ideal(response_4, ideal_4)
print(f"Result for Case 4: {result_4}")  # Output: -1 (placeholder for further evaluation)

In [ ]:
import json

def eval_response_with_ideal(response, ideal):
    # Replace single quotes with double quotes for valid JSON
    json_like_str = response.replace("'", '"')

    # Parse into a list of dictionaries
    try:
        l_of_d = json.loads(json_like_str)
    except json.JSONDecodeError:
        print("Invalid JSON format in response.")
        return 0  # Return 0 for invalid response format

    # Check if both the parsed response and ideal output are empty
    if l_of_d == [] and ideal == []:
        return 1  # Perfect match: both are empty
    
    # Check if either the parsed response or ideal output is empty
    elif l_of_d == [] or ideal == []:
        return 0  # Mismatch: one is empty while the other is not

    correct = 0    

    # Further evaluation logic can go here...
    
    return correct / len(l_of_d) if l_of_d else 0  # Placeholder return value

# Example usage:

# Case where both response and ideal have data
response = "[{'category': 'Computers and Laptops', 'products': ['TechPro Ultrabook', 'BlueWave Gaming Laptop']}, {'category': 'Gaming Consoles', 'products': ['GameSphere X']}]"
ideal = {
    'Computers and Laptops': ['TechPro Ultrabook', 'BlueWave Gaming Laptop', 'PowerLite Convertible'],
    'Gaming Consoles': ['GameSphere X']
}

result = eval_response_with_ideal(response, ideal)
print("Evaluation Result:", result)  # Output will depend on further evaluation logic


#####  eval_response_with_ideal

1.Initializing a counter for correct matches.
2.Printing debug information if enabled.
3.Iterating through each item in the parsed response.
4.Checking for valid categories and products.
5.Comparing sets of products from both actual and ideal outputs.
6.Calculating and returning an accuracy percentage based on matches.

In [29]:
# Example AI-generated response (string format)
response = "[{'category': 'Computers and Laptops', 'products': ['TechPro Ultrabook', 'BlueWave Gaming Laptop']}]"

# Example ideal output (dictionary format)
ideal = {
    'Computers and Laptops': ['TechPro Ultrabook', 'BlueWave Gaming Laptop', 'PowerLite Convertible'],
    'Gaming Consoles': ['GameSphere X']
}

# Call the evaluation function
result = eval_response_with_ideal(response, ideal, debug=True)

print("Evaluation Result:", result)

response
[{'category': 'Computers and Laptops', 'products': ['TechPro Ultrabook', 'BlueWave Gaming Laptop']}]
l_of_d is
[{'category': 'Computers and Laptops', 'products': ['TechPro Ultrabook', 'BlueWave Gaming Laptop']}]
prod_set
 {'BlueWave Gaming Laptop', 'TechPro Ultrabook'}

prod_set_ideal
 {'BlueWave Gaming Laptop', 'PowerLite Convertible', 'TechPro Ultrabook'}
incorrect
prod_set: {'BlueWave Gaming Laptop', 'TechPro Ultrabook'}
prod_set_ideal: {'BlueWave Gaming Laptop', 'PowerLite Convertible', 'TechPro Ultrabook'}
response is a subset of the ideal answer
Evaluation Result: 0.0


In [ ]:
import json

def eval_response_with_ideal(response, ideal, debug=False):
    # Parse the response string into a list of dictionaries
    json_like_str = response.replace("'", '"')  # Replace single quotes with double quotes for valid JSON
    l_of_d = json.loads(json_like_str)  # Parse into a list of dictionaries

    correct = 0  # 1. Initializing a counter for correct matches.
    
    if debug:  # 2. Printing debug information if enabled.
        print("l_of_d is")
        print(l_of_d)
    
    for d in l_of_d:  # 3. Iterating through each item in the parsed response.
        cat = d.get('category')  # 4. Checking for valid categories and products.
        prod_l = d.get('products')
        
        if cat and prod_l:
            # Convert list to set for comparison
            prod_set = set(prod_l)
            # Get ideal set of products
            ideal_cat = ideal.get(cat)
            
            if ideal_cat:
                prod_set_ideal = set(ideal.get(cat))
            else:
                if debug:
                    print(f"did not find category {cat} in ideal")
                    print(f"ideal: {ideal}")
                continue
            
            if debug:  # 5. Comparing sets of products from both actual and ideal outputs.
                print("prod_set\n", prod_set)
                print()
                print("prod_set_ideal\n", prod_set_ideal)

            if prod_set == prod_set_ideal:
                if debug:
                    print("correct")
                correct += 1
            else:
                print("incorrect")
                print(f"prod_set: {prod_set}")
                print(f"prod_set_ideal: {prod_set_ideal}")
                
                if prod_set <= prod_set_ideal:
                    print("response is a subset of the ideal answer")
                elif prod_set >= prod_set_ideal:
                    print("response is a superset of the ideal answer")

    # Count correct over total number of items in list
    pc_correct = correct / len(l_of_d) if l_of_d else 0  # Calculating and returning an accuracy percentage based on matches.
    
    return pc_correct

# Example usage:

# Simulated AI-generated response (string format)
response = "[{'category': 'Computers and Laptops', 'products': ['TechPro Ultrabook', 'BlueWave Gaming Laptop']}, {'category': 'Gaming Consoles', 
'products': ['GameSphere X']}]"

# Example ideal output (dictionary format)
ideal = {
    'Computers and Laptops': ['TechPro Ultrabook', 'BlueWave Gaming Laptop', 'PowerLite Convertible'],
    'Gaming Consoles': ['GameSphere X']
}

# Call the evaluation function with debug enabled
result = eval_response_with_ideal(response, ideal, debug=True)

# Print the final evaluation result
print("Evaluation Result:", result)